In [1]:
import pandas as pd 
import numpy as np
import math 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim

import pickle 
import main
from main import add_noise
from main import denoise

Using cuda device


In [2]:
with open('data/sampled_filters_train.pkl', 'rb') as f:
    data = pickle.load(f)

In [7]:
data['zmin']

dict_keys(['wavelengths', 'spectra', 'X', 'y', 'z', 'zmin', 'zmax', 'filter_names'])

In [32]:

def get_data(data):
    #in data
    #wavelength (4000000, 149)
    #spectra (4000000, 149)
    #X (4000000, 37)
    #y (4000000, 4)
    #z (4000000, 1)
    #zmin (float)
    #zmax (float)
    #filter_names (string list), len = 37
    df_X = pd.DataFrame(data['X'])
    outlier_columns = df_X.columns[df_X.gt(58.8).any(axis = 0)]
    df_X = df_X.drop(outlier_columns, axis = 1)
    X = df_X.to_numpy()
    y = pd.DataFrame(data['y'])
    z = pd.DataFrame(data['z'])
    wavelengths = data['wavelengths']
    spectra = data['spectra']
    
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(df_X)

    names=data['filter_names']
    filter_wls = [int(name[1:-1].rstrip('W')) for name in names]
    filter_wls = [element for i, element in enumerate(filter_wls) if i not in outlier_columns]
    indices=[0,10,30,500,15000,800000]
    return filter_wls, wavelengths, spectra, X, X_normalized, y, z

In [33]:
filter_wls, wavelengths, spectra, X, X_normalized, y, z = get_data(data)
X[0]

array([20.786884, 20.960768, 21.265028, 21.509062, 21.601727, 21.808668,
       21.899237, 21.870485, 22.042837, 22.1664  , 22.201782, 22.337282,
       22.382853, 22.393621, 22.491522, 22.527351, 22.557444, 22.61449 ,
       22.445515, 22.539598, 22.658358, 22.670202, 22.752363, 22.814674,
       22.803913, 23.091652, 23.25525 , 23.325388, 23.654198, 24.586325,
       24.331484], dtype=float32)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
model_big = main.BigUNet().double().to(device)
model_big.load_state_dict(torch.load('output/model_param_model_big.pt'))

model_medBig = main.MedBigUNet().double().to(device)
model_medBig.load_state_dict(torch.load('output/model_param_model_medBig.pt'))

model_medium = main.MediumUNet().double().to(device)
model_medium.load_state_dict(torch.load('output/model_param_model_medium.pt'))

model_small = main.SmallUNet().double().to(device)
model_small.load_state_dict(torch.load('output/model_param_model_small.pt'))

modelList = [model_big, model_medBig, model_medium, model_small]
modelNames = ["model_big", "model_medBig", "model_medium", "model_small"]